# 시각 심화

- **Instructor**: Jongwoo Lim / Jiun Bae
- **Email**: [jlim@hanyang.ac.kr](mailto:jlim@hanyang.ac.kr) / [jiunbae.623@gmail.com](mailto:jiunbae.623@gmail.com)

## NeuralNetwork Example

In this example you will practice a simple neural network written by only [Numpy](https://www.numpy.org) which is fundamental package for scientific computing with Python. The goals of this example are as follows:

- Understand **Neural Networks** and how they work.
- Learn basically how to **write and use code**(*Numpy*).

*If you are more familiar with PyTorch and TensorFlow(or Keras), You might be wondering why to write from the ground up with numpy instead of the built-in framework. This process is essential for understanding how a neural network works, and if you understand it, will not be too difficult to write in code.*

And this example also is written in [IPython Notebook](https://ipython.org/notebook.html), an interactive computational environment, in which you can run code directly.

### Environments

In this assignment, we assume the follows environments. 

The [Python](https://www.python.org) is a programming language that lets you work quickly and integrate systems more effectively. It is widely used in various fields, and also used in machine learning.

The [Pytorch](https://pytorch.org) is an open source deep learning platform, provides a seamless path from research to production.

The [Tensorflow](https://www.tensorflow.org) is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community resources that lets researchers push the state-of-the-art in ML and developers easily build and deploy ML powered applications.

The [CUDA®](https://developer.nvidia.com/cuda-zone) Toolkit provides high-performance GPU-accelerated computation. In deep learning, the model takes an age to train without GPU-acceleration. ~~even with the GPU, it still takes a lot of time~~.


- [Python3](https://www.python.org/downloads/) (recommend 3.6 or above)
- [PyTorch](https://pytorch.org) (recommend 1.0)
- [Tensorflow](https://tensorflow.org) (recommend above 1.13.0, but under 2.0 *There are huge difference between 2.0 and below*)
- [NumPy](http://www.numpy.org) the fundamental package for scientific computing with Python


- (Optional) [Anaconda](https://www.anaconda.com/distribution/#download-section), *popular Python Data Science Platform*
- (Optional) [Jupyter](https://jupyter.org/) (Notebook or Lab)
- (Optional) [CUDA](https://developer.nvidia.com/cuda-downloads) support GPU


Python packages can install by `pip install [package name]` or using **Anaconda** by `conda install [package name]`.

*If you are having trouble installing or something else, please contact TA or jiun.maydev@gmail.com.*

# Code

### Import packages

Numpy the basic scientific computing package used in customary.

In [1]:
import numpy as np

## Load MNIST dataset

Tesnorflow provide mnist dataset as binary archive file [link](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/tutorials/mnist/download/index.md).
In this exampe, we already downloaded datafile in `./data` directory. So just laod dataset from `./data`.

In [2]:
import numpy as np
from tensorflow.python import keras

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data('mnist.npz')

## (Optional) Visualize

In [4]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))

In [5]:
for image, label, _ in zip(train_images, train_labels, range(10)):
    print(label)
    show(image.reshape((28, 28)))

5


0


4


1


9


2


1


3


1


4


## Preprocessing

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255. We scale these values to a range of 0 to 1 before feeding to the neural network model. For this, we divide the values by 255. It's important that the training set and the testing set are preprocessed in the same way:

In [6]:
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

train_images = train_images / 255.
test_images = test_images / 255.

In [7]:
from tensorflow.python.keras.utils import to_categorical
num_classes = 10
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

## Network

This is a simple two dense(fully connected) layer network. The code is quite easy.

So, whole network architecture as follow:

- Dense
- Sigmoid
- Dense
- Sigmoid
- Dense
- Sigmoid

### Dense Layer

In [8]:
class Layer:
    pass

class Dense(Layer):
    def __init__(self, input_units, output_units):
        self.weights = np.random.randn(output_units, input_units) * .01
        self.biases = np.random.randn(output_units, 1) * .1
        
    def forward(self, inputs):
        self.inputs = inputs
        
        return np.dot(self.weights, inputs) + self.biases
      
    def backward(self, grads):
        size = np.size(grads, -1)

        self.grad_weights = np.dot(grads, self.inputs.T) / size
        self.grad_biases = np.sum(grads, axis=1, keepdims=True) / size

        return np.dot(self.weights.T, grads)

    def update(self, lr: float = .01):
        # Here we perform a stochastic gradient descent step.
        self.weights = self.weights - lr * self.grad_weights
        self.biases = self.biases - lr * self.grad_biases

### Activate Function: Sigmoid

In [9]:
class Sigmoid(Layer):
    def forward(self, inputs):
        self.inputs = inputs
        return 1. / (1. + np.exp(-inputs))

    def backward(self, grads):
        r = self.forward(self.inputs)
        return grads * r * (1. - r)
    
    def update(self, lr):
        pass

### Training

In [22]:
from typing import List
from functools import reduce


def fit(networks: List[Layer], X, y, train=True, epsilon=1e-7):
    X = np.reshape(X, (X.shape[0], -1))
    
    # Forward
    preds = reduce(lambda inputs, layer: layer.forward(inputs), [X.T, *networks]).T
    
    # Compute Loss
    loss = -np.sum(y * np.log(np.clip(preds, epsilon, 1. - epsilon) + epsilon)) / np.size(preds, 0)
    
    if train:
        # Backward
        grads = -(np.divide(y, preds) - np.divide(1 - y, 1 - preds))
        grads = reduce(lambda grads, layer: layer.backward(grads), [grads.T, *reversed(networks)])
    
        # Update
        for layer in networks:
            layer.update(lr)
    
    return loss.mean(), preds

## Prepare

In [17]:
def get_batch(datasets, batch):
    image, label = None, None
    images, labels = datasets
    for b, (i, l) in enumerate(zip(*datasets)):
        if not (b % batch):
            if image is not None and label is not None:
                yield image, label
            image = np.empty((batch, 28, 28, 1), dtype=np.float32)
            label = np.empty((batch, 10), dtype=np.uint8)
            
        image[b % batch] = i
        label[b % batch] = l

In [18]:
np.random.seed(42)

In [19]:
lr = .1
batch = 128
epochs = 10

In [20]:
network = [
    Dense(28*28, 100),
    Sigmoid(),
    Dense(100, 200),
    Sigmoid(),
    Dense(200, 10),
    Sigmoid(),
]

## Train

In [23]:
for epoch in range(epochs):
    # Train scope
    train_loss, test_loss, test_acc = 0, 0, 0
    for images, labels in get_batch((train_images, train_labels), batch):
        loss, _ = fit(network, images, labels)
        train_loss += loss
        
    for images, labels in get_batch((test_images, test_labels), batch):
        loss, preds = fit(network, images, labels, train=False)

        test_loss += loss
        test_acc += (preds.argmax(axis=-1) == labels.argmax(axis=-1)).mean()
    print(f'Epoch: {epoch}')
    print(f'\tTrain Loss: {train_loss / (len(train_images) / batch)}')
    print(f'\tTest Loss: {test_loss / (len(test_images) / batch)}')
    print(f'\tTest Acc: {test_acc / (len(test_images) / batch)}')

Epoch: 0
	Train Loss: 2.305125801856038
	Test Loss: 2.2979551816602273
	Test Acc: 0.103
Epoch: 1
	Train Loss: 2.3044144317430537
	Test Loss: 2.297525418632348
	Test Acc: 0.103
Epoch: 2
	Train Loss: 2.3033433392527867
	Test Loss: 2.2963360867401703
	Test Acc: 0.103
Epoch: 3
	Train Loss: 2.299853396264826
	Test Loss: 2.288329348381132
	Test Acc: 0.1902
Epoch: 4
	Train Loss: 2.219474797138108
	Test Loss: 1.9916116627237839
	Test Acc: 0.3034
Epoch: 5
	Train Loss: 1.750021704120589
	Test Loss: 1.458148136785472
	Test Acc: 0.472
Epoch: 6
	Train Loss: 1.3727509055175149
	Test Loss: 1.2419250145996898
	Test Acc: 0.6406
Epoch: 7
	Train Loss: 1.082631134529613
	Test Loss: 1.0283236629227681
	Test Acc: 0.7076
Epoch: 8
	Train Loss: 0.9477122327513593
	Test Loss: 0.9220482727452977
	Test Acc: 0.7624
Epoch: 9
	Train Loss: 0.825922282929919
	Test Loss: 0.7835602915349938
	Test Acc: 0.8152


## Test

In [24]:
for image, label, _ in zip(test_images, test_labels, range(10)):
    show((image[:, :, 0] * 255.).astype(np.uint8))
    prediction = reduce(lambda inputs, layer: layer.forward(inputs), [np.reshape(np.array(image), -1)[None, :].T, *network])
    print (f'Label: {label.argmax(-1)}, Prediction: {prediction.argmax()}')

Label: 7, Prediction: 7


Label: 2, Prediction: 2


Label: 1, Prediction: 1


Label: 0, Prediction: 0


Label: 4, Prediction: 4


Label: 1, Prediction: 1


Label: 4, Prediction: 9


Label: 9, Prediction: 9


Label: 5, Prediction: 6


Label: 9, Prediction: 9
